# OTX API Usage for Advanced-API User

### Requirements
R3. Advanced-level Python API users provide a Dataset, and a model type (str) as input, and OTX provides a workflow.


In [1]:
data_root = "../../tests/assets/classification_dataset"

In [2]:
from otx.engine import Engine

engine = Engine(
    data_root=data_root,
    task="MULTI_CLASS_CLS",
    model="efficientnet_b0_light",
)

/home/harimkan/workspace/repo/otx-fork/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip


In [3]:
engine.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                         | Params
-------------------------------------------------------------
0 | model       | MMPretrainMulticlassClsModel | 4.0 M 
1 | val_metric  | MulticlassAccuracy           | 0     
2 | test_metric | MulticlassAccuracy           | 0     
-------------------------------------------------------------
4.

/home/harimkan/workspace/repo/otx-fork/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 9: 100%|██████████| 1/1 [00:00<00:00, 10.71it/s, v_num=9, train/loss=0.322, val/accuracy=1.000]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:00<00:00,  7.02it/s, v_num=9, train/loss=0.322, val/accuracy=1.000]


{'train/loss': tensor(0.3222), 'val/accuracy': tensor(1.)}


R4. Advanced-level Python API users provide a Dataset, and Model (nn.Module) as input, and OTX provides the workflow.

In [4]:
import torch
from torch import nn
from torchvision.models.resnet import ResNet50_Weights, resnet50


class ResNet50WithLossComputation(nn.Module):
    def __init__(self, num_classes: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        net = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        net.fc = nn.Linear(
            in_features=net.fc.in_features, out_features=self.num_classes,
        )
        self.net = net
        self.softmax = nn.Softmax(dim=-1)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, images: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        logits = self.net(images)

        if self.training:
            return self.criterion(logits, labels)

        return self.softmax(logits)


In [5]:
from typing import Any

import torch
from otx.core.data.entity.base import OTXBatchLossEntity
from otx.core.data.entity.classification import (
    MulticlassClsBatchDataEntity,
    MulticlassClsBatchPredEntity,
)
from otx.core.model.entity.classification import OTXMulticlassClsModel
from torch import nn


class OTXResNet50(OTXMulticlassClsModel):
    def __init__(self, num_classes: int) -> None:
        self.num_classes = num_classes
        super().__init__()
        self.register_buffer(
            "mean",
            torch.FloatTensor([123.675, 116.28, 103.53]).view(-1, 1, 1),
            False,
        )
        self.register_buffer(
            "std",
            torch.FloatTensor([58.395, 57.12, 57.375]).view(-1, 1, 1),
            False,
        )

    def _create_model(self) -> nn.Module:
        # ResNet50_Weights.IMAGENET1K_V2 is a really powerful pretrained model equipped with the modern training scheme:
        # ImageNet-1K acc@1: 80.858, acc@5": 95.434.
        return ResNet50WithLossComputation(num_classes=self.num_classes)

    def _customize_inputs(self, inputs: MulticlassClsBatchDataEntity) -> dict[str, Any]:
        images = torch.stack((*inputs.images,), dim=0).to(dtype=torch.float32)
        images = (images - self.mean) / self.std
        return {
            "images": images,
            "labels": torch.cat(inputs.labels, dim=0),
        }

    def _customize_outputs(
        self, outputs: Any, inputs: MulticlassClsBatchDataEntity
    ) -> MulticlassClsBatchPredEntity | OTXBatchLossEntity:
        if self.training:
            return {"loss": outputs}

        # To list, batch-wise
        scores = torch.unbind(outputs, 0)

        return MulticlassClsBatchPredEntity(
            batch_size=inputs.batch_size,
            images=inputs.images,
            imgs_info=inputs.imgs_info,
            scores=scores,
            labels=inputs.labels,
        )

In [6]:
from otx.engine import Engine

engine = Engine(
    data_root=data_root,
    work_dir="./otx-workspace",
    device="gpu",
    model=OTXResNet50(num_classes=2),
)

In [7]:
engine.train(max_epochs=3)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type               | Params
---------------------------------------------------
0 | model       | OTXResNet50        | 23.5 M
1 | val_metric  | MulticlassAccuracy | 0     
2 | test_metric | MulticlassAccuracy | 0     
---------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)


Epoch 2: 100%|██████████| 1/1 [00:00<00:00,  8.74it/s, v_num=10, train/loss=0.698, val/accuracy=1.000]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s, v_num=10, train/loss=0.698, val/accuracy=1.000]


{'train/loss': tensor(0.6983), 'val/accuracy': tensor(1.)}

User provide optimizer & scheduler

In [8]:
from functools import partial

from otx.engine import Engine
from torch.optim.adam import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

engine = Engine(
    data_root=data_root,
    work_dir="./otx-workspace",
    device="gpu",
    model=OTXResNet50(num_classes=2),
    optimizer=partial(Adam, lr=1e-4),
    scheduler=partial(CosineAnnealingLR, T_max=10),
)

R5. Advanced-level Python API users can use custom training using all the trainer parameters available in lightning.

In [9]:
engine.train(
    max_epochs=3,
    precision="16",
)

/home/harimkan/workspace/repo/otx-fork/venv/lib/python3.10/site-packages/lightning/fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type               | Params
---------------------------------------------------
0 | model       | OTXResNet50        | 23.5 M
1 | val_metric  | MulticlassAccuracy | 0     
2 | test_metric | MulticlassAccuracy | 0     
---------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)


/home/harimkan/workspace/repo/otx-fork/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  8.26it/s, v_num=11, train/loss=0.708]

/home/harimkan/workspace/repo/otx-fork/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 11.50it/s, v_num=11, train/loss=0.716, val/accuracy=1.000]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:00<00:00,  4.80it/s, v_num=11, train/loss=0.716, val/accuracy=1.000]


{'train/loss': tensor(0.7165), 'val/accuracy': tensor(1.)}